In [ ]:
!pip install fast-mail-parser
!pip install tabulate

In [1]:
import yaml

def prompt_loader(path: str) -> str:
    """
    The function `prompt_loader` reads and returns the content of a file specified by the input path.
    
    :param path: The `path` parameter in the `prompt_loader` function is a string that represents the
    file path to the prompt file that you want to load. This function reads the content of the file
    located at the specified path and returns it as a string
    :type path: str
    :return: The function `prompt_loader` returns a string containing the content loaded from the file
    located at the specified path.
    """
    prompt = ""
    with open(path, 'r') as file:
        prompt = yaml.safe_load(file)

    return prompt

In [2]:
import subprocess

def run_command(command, cwd=None):
    """Run a shell command."""
    result = subprocess.run(command, shell=True, text=True, cwd=cwd, capture_output=True)
    if result.returncode == 0:
        print(result.stdout)
    else:
        print(f"Error: {result.stderr}")
        raise Exception("Command failed")
    return result

def git_add_commit_push(commit_message, branch_name="main"):
    """Automate git add, commit, and push."""
    try:
        # Navigate to the repository
        print("Navigating to the repository...")
        
        # Run git add
        print("Adding files...")
        run_command("git add .")

        # Run git commit
        print("Committing changes...")
        run_command(f'git commit -m "{commit_message}"')

        # Run git push
        print("Pushing to remote...")
        run_command(f"git push origin {branch_name}")

        print("All changes have been pushed successfully!")

    except Exception as e:
        print(f"An error occurred: {e}")


In [3]:
import sys
import enum
from os import listdir
from os.path import isfile, join
import pandas
from fast_mail_parser import parse_email, ParseError
from datetime import datetime
import typing_extensions as typing
from llm_solution import LLMSolution


class ContentType(enum.Enum):
    COURSE = "course"
    TUTORIAL = "tutorial"
    BOOK = "book"
    PRODUCTorTOOL = "product_or_tool"
    CODE = "code"
    NEW_MODEL = "new_model"
    FUNDING = "funding"

class ContentEntry(typing.TypedDict):
    content_type: ContentType
    description: str
    link: str
    tag: list[str]

client_provider = "gemini"
model_name = "gemini-2.0-flash-exp"
llm_client = LLMSolution(llm_client=client_provider, model_name=model_name)

summarize_prompt = prompt_loader("summarize.yaml")

email_path = "emails/"
done_file_name = 'done.txt'
onlyfiles = [join(email_path, f) for f in listdir(email_path) if isfile(join(email_path, f))]

with open(done_file_name) as file:
    already_done_files = [line.rstrip() for line in file] 

done_file = open(done_file_name, 'a')

output_table = []
for filename in onlyfiles:
    if filename in already_done_files:
        continue
    with open(filename, 'r') as f:
        message_payload = f.read()

    try:
        email = parse_email(message_payload)
    except ParseError as e:
        print("Failed to parse email: ", e)
        sys.exit(1)


    print("Processing data for generation...")
    system_prompt = summarize_prompt['system_prompt']
    args = {'args': email.text_html} 
    user_prompt = llm_client.enrich_prompt(prompt=summarize_prompt['user_prompt'], args=args)
    current_table = llm_client.create_response(system_prompt=system_prompt, user_prompt=user_prompt, response_format=ContentEntry)
    output_table.extend(current_table)

    done_file.write(filename + "\n")

done_file.close()


df = pandas.DataFrame.from_dict(output_table)#.rename(columns={'map_name':'Name', 'nan':'NaN', 'packets':'Packages'})
markdown_content = df.to_markdown(index=False)#, tablefmt='fancy_grid')


today_date = datetime.today().strftime('%Y-%m-%d')
with open("README.md", 'a') as file:
    markdown_content = "\n\n## Added on " + str(today_date) + "\n" + markdown_content
    file.write(markdown_content + "\n\n")

commit_message = "Added new course content"
branch_name = "main"  # Replace with the branch name if different
git_add_commit_push(commit_message, branch_name)

c:\Users\asankaran\AppData\Local\anaconda3\envs\chesspy310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing gemini client with model: gemini-2.0-flash-exp ... 
Processing data for generation...
Processing data for generation...
Processing data for generation...
Processing data for generation...
Processing data for generation...
Processing data for generation...
Processing data for generation...
Processing data for generation...
Navigating to the repository...
Adding files...

Committing changes...
[main 6381aa5] Added new course content
 Committer: Anush Sankaran <asankaran@microsoft.com>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 11 files changed, 14194 insertions(+), 1 deletion(-)
 create mode 100644 "emails/Applied AI_ More C